#Fábrica de Cookie

In [ ]:
!pip install simpy
import simpy
import numpy as np
import random

In [ ]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    RED = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

TEMPO_SIMULACAO = 50

#Arquitetura 1 - fila única e múltiplos Fornos

In [ ]:
fornos_status = [False, False, False, False]  # False = livre

def pegar_forno_livre():
    for i, ocupado in enumerate(fornos_status):
        if not ocupado:
            fornos_status[i] = True
            return i
    return None


def liberar_forno(i):
    fornos_status[i] = False


def processar_cookie_1(env, fornos, nome_do_lote, tempo_de_preparo):
    # tempo antes de chegar ao forno
    yield env.timeout(tempo_de_preparo)

    # verifica se vai entrar na fila
    if fornos.count == fornos.capacity:
        print(f"{bcolors.WARNING}[{env.now:3d}] Lote {nome_do_lote} entrou na FILA (todos os fornos ocupados){bcolors.ENDC}")
    else:
        print(f"{bcolors.OKCYAN}[{env.now:3d}] Lote {nome_do_lote} chegou e há forno LIVRE{bcolors.ENDC}")

    # requisita um dos fornos (fila única)
    req = fornos.request()
    yield req

    # agora escolhe qual forno realmente pegou
    forno_id = pegar_forno_livre()

    print(f"[{env.now:3d}] Lote {nome_do_lote} entrou no FORNO {forno_id}")

    # tempo dentro do forno
    yield env.timeout(tempo_de_preparo)

    print(f"{bcolors.OKGREEN}[{env.now:3d}] 🍪 Lote {nome_do_lote} saiu do FORNO {forno_id}{bcolors.ENDC}")

    # libera forno e o recurso
    liberar_forno(forno_id)
    fornos.release(req)

In [ ]:
def gerar_lotes_1(env, fornos):
    lote_id = 0
    while True:
        lote_id += 1
        tempo_preparo = random.randint(3, 8)
        env.process(processar_cookie_1(env, fornos, f"L{lote_id}", tempo_preparo))
        yield env.timeout(random.randint(1, 4))

In [ ]:
env = simpy.Environment()

# 4 FORNOS
fornos = simpy.Resource(env, capacity=4)

env.process(gerar_lotes_1(env, fornos))
env.run(until=TEMPO_SIMULACAO)

[  4] Lote L1 chegou e há forno LIVRE
[  4] Lote L1 entrou no FORNO 0
[  8] 🍪 Lote L1 saiu do FORNO 0
[ 10] Lote L3 chegou e há forno LIVRE
[ 10] Lote L3 entrou no FORNO 0
[ 11] Lote L2 chegou e há forno LIVRE
[ 11] Lote L2 entrou no FORNO 1
[ 13] 🍪 Lote L3 saiu do FORNO 0
[ 15] Lote L4 chegou e há forno LIVRE
[ 15] Lote L4 entrou no FORNO 0
[ 18] 🍪 Lote L2 saiu do FORNO 1
[ 18] Lote L7 chegou e há forno LIVRE
[ 18] Lote L7 entrou no FORNO 1
[ 20] Lote L5 chegou e há forno LIVRE
[ 20] Lote L6 chegou e há forno LIVRE
[ 20] Lote L8 entrou na FILA (todos os fornos ocupados)
[ 20] Lote L5 entrou no FORNO 2
[ 20] Lote L6 entrou no FORNO 3
[ 21] 🍪 Lote L7 saiu do FORNO 1
[ 21] Lote L8 entrou no FORNO 1
[ 22] 🍪 Lote L4 saiu do FORNO 0
[ 24] Lote L9 chegou e há forno LIVRE
[ 24] 🍪 Lote L8 saiu do FORNO 1
[ 24] Lote L9 entrou no FORNO 0
[ 26] 🍪 Lote L6 saiu do FORNO 3
[ 28] 🍪 Lote L5 saiu do FORNO 2
[ 28] 🍪 Lote L9 saiu do FORNO 0
[ 29] Lote L10 chegou e há forno LIVRE
[ 29] Lote L11 chegou e h

# Arquitetura 2 - múltiplas filas, múltiplos fornos.

In [ ]:
def processar_cookie_2(env, forno, nome_do_lote, tempo_de_preparo, forno_id):
    # tempo antes de chegar ao forno
    yield env.timeout(tempo_de_preparo)

    # se o forno estiver ocupado → fila
    if forno.count == forno.capacity:
        print(f"{bcolors.WARNING}[{env.now:3d}] Lote {nome_do_lote} ENTROU na FILA do forno {forno_id}{bcolors.ENDC}")
    else:
        print(f"{bcolors.OKCYAN}[{env.now:3d}] Lote {nome_do_lote} chegou e o forno {forno_id} está LIVRE{bcolors.ENDC}")

    req = forno.request()
    yield req

    print(f"[{env.now:3d}] Lote {nome_do_lote} entrou no FORNO {forno_id}")

    yield env.timeout(tempo_de_preparo)

    print(f"{bcolors.OKGREEN}[{env.now:3d}] 🍪 Lote {nome_do_lote} saiu do FORNO {forno_id}{bcolors.ENDC}")

    forno.release(req)


In [ ]:
def gerar_lotes_2(env, fornos):
    lote_id = 0
    while True:
        lote_id += 1
        tempo_preparo = random.randint(3, 8)

        # escolha do forno (o de menor fila)
        # escolha do forno (menor fila)
        filas = [f.queue for f in fornos]
        forno_escolhido = min(range(len(fornos)), key=lambda i: len(filas[i]))


        env.process(
            processar_cookie_2(
                env,
                fornos[forno_escolhido],
                f"L{lote_id}",
                tempo_preparo,
                forno_escolhido
            )
        )

        yield env.timeout(random.randint(1, 4))

In [ ]:
env = simpy.Environment()

# 4 fornos → 4 filas distintas
fornos = [simpy.Resource(env, capacity=1) for _ in range(4)]

env.process(gerar_lotes_2(env, fornos))

env.run(until=50)


[  8] Lote L1 chegou e o forno 0 está LIVRE
[  8] Lote L1 entrou no FORNO 0
[  9] Lote L2 ENTROU na FILA do forno 0
[ 13] Lote L3 ENTROU na FILA do forno 0
[ 13] Lote L4 ENTROU na FILA do forno 0
[ 16] 🍪 Lote L1 saiu do FORNO 0
[ 16] Lote L2 entrou no FORNO 0
[ 19] Lote L5 chegou e o forno 1 está LIVRE
[ 19] Lote L5 entrou no FORNO 1
[ 22] Lote L6 ENTROU na FILA do forno 1
[ 22] 🍪 Lote L2 saiu do FORNO 0
[ 22] Lote L3 entrou no FORNO 0
[ 24] Lote L7 ENTROU na FILA do forno 1
[ 27] 🍪 Lote L5 saiu do FORNO 1
[ 27] Lote L6 entrou no FORNO 1
[ 28] Lote L8 chegou e o forno 2 está LIVRE
[ 28] Lote L8 entrou no FORNO 2
[ 30] 🍪 Lote L3 saiu do FORNO 0
[ 30] Lote L9 ENTROU na FILA do forno 2
[ 30] Lote L4 entrou no FORNO 0
[ 33] Lote L10 ENTROU na FILA do forno 2
[ 33] 🍪 Lote L8 saiu do FORNO 2
[ 33] Lote L9 entrou no FORNO 2
[ 34] 🍪 Lote L6 saiu do FORNO 1
[ 34] Lote L7 entrou no FORNO 1
[ 35] 🍪 Lote L4 saiu do FORNO 0
[ 36] Lote L11 ENTROU na FILA do forno 2
[ 36] Lote L13 chegou e o forno 0 

# Arquitetura 3 - múltiplas filas e um forno

In [ ]:
def processar_cookie_3(env, forno, nome_do_lote, tempo_de_preparo, forno_id):
    # tempo antes de chegar ao forno
    yield env.timeout(tempo_de_preparo)

    # se o forno estiver ocupado → fila
    if forno.count == forno.capacity:
        print(f"{bcolors.WARNING}[{env.now:3d}] Lote {nome_do_lote} ENTROU na FILA do forno {forno_id}{bcolors.ENDC}")
    else:
        print(f"{bcolors.OKCYAN}[{env.now:3d}] Lote {nome_do_lote} chegou e o forno {forno_id} está LIVRE{bcolors.ENDC}")

    req = forno.request()
    yield req

    print(f"[{env.now:3d}] Lote {nome_do_lote} entrou no FORNO {forno_id}")

    yield env.timeout(tempo_de_preparo)

    print(f"{bcolors.OKGREEN}[{env.now:3d}] 🍪 Lote {nome_do_lote} saiu do FORNO {forno_id}{bcolors.ENDC}")

    forno.release(req)


In [ ]:
def gerar_lotes_3(env, fornos):
    lote_id = 0
    while True:
        lote_id += 1
        tempo_preparo = random.randint(3, 8)

        env.process(
            processar_cookie_3(
                env,
                fornos[0],
                f"L{lote_id}",
                tempo_preparo,
                0
            )
        )

        yield env.timeout(random.randint(1, 4))


In [ ]:
env = simpy.Environment()

# 1 fornos
fornos = [simpy.Resource(env, capacity=1) for _ in range(1)]

env.process(gerar_lotes_3(env, fornos))

env.run(until=50)


[  6] Lote L2 chegou e o forno 0 está LIVRE
[  6] Lote L2 entrou no FORNO 0
[  8] Lote L1 ENTROU na FILA do forno 0
[  9] 🍪 Lote L2 saiu do FORNO 0
[  9] Lote L1 entrou no FORNO 0
[ 10] Lote L3 ENTROU na FILA do forno 0
[ 14] Lote L4 ENTROU na FILA do forno 0
[ 17] 🍪 Lote L1 saiu do FORNO 0
[ 17] Lote L3 entrou no FORNO 0
[ 18] Lote L5 ENTROU na FILA do forno 0
[ 18] Lote L6 ENTROU na FILA do forno 0
[ 20] 🍪 Lote L3 saiu do FORNO 0
[ 20] Lote L4 entrou no FORNO 0
[ 23] 🍪 Lote L4 saiu do FORNO 0
[ 23] Lote L5 entrou no FORNO 0
[ 24] Lote L7 ENTROU na FILA do forno 0
[ 25] Lote L8 ENTROU na FILA do forno 0
[ 27] Lote L9 ENTROU na FILA do forno 0
[ 29] 🍪 Lote L5 saiu do FORNO 0
[ 29] Lote L6 entrou no FORNO 0
[ 30] Lote L10 ENTROU na FILA do forno 0
[ 32] 🍪 Lote L6 saiu do FORNO 0
[ 32] Lote L7 entrou no FORNO 0
[ 33] Lote L11 ENTROU na FILA do forno 0
[ 36] Lote L12 ENTROU na FILA do forno 0
[ 37] 🍪 Lote L7 saiu do FORNO 0
[ 37] Lote L8 entrou no FORNO 0
[ 38] Lote L13 ENTROU na FILA do 